In [ ]:
#pydicom instalation
conda install -c conda-forge pydicom

In [ ]:
!conda install -c conda-forge nodejs  # or some other way to have a recent node
#installing node with this command led to some version incompatible problems so i used this one instead:
!conda install nodejs -c conda-forge --repodata-fn=repodata.json
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install ipyvolume
!jupyter labextension install jupyter-threejs

In [ ]:
! conda install -c conda-forge gdcm

In [2]:
import ipyvolume as ipv
import pydicom
import numpy as np
import osc


files = []
path = '/home/pariya/S0000001119/'
for s in os.listdir(path):
    print("loading {}".format(s))
    files.append(pydicom.dcmread('/home/pariya/S0000001119/' + s))
print("file count: {}".format(len(files)))


# skip files with no SliceLocation (eg scout views)
slices = []
skipcount = 0
for f in files:
    if hasattr(f, 'SliceLocation'):
        slices.append(f)
    else:
        skipcount = skipcount + 1

print("skipped, no SliceLocation: {}".format(skipcount))
sh = slices[0].pixel_array.shape
removed=[]
for s in slices:
    if sh!=s.pixel_array.shape:
        removed.append(s)
for r in removed:
    slices.remove(r)
# ensure they are in the correct order
slices = sorted(slices, key=lambda s: s.SliceLocation)

# pixel aspects, assuming all slices are the same
ps = slices[0].PixelSpacing
ss = slices[0].SliceThickness
ax_aspect = ps[1] / ps[0]
sag_aspect = ps[1] / ss
cor_aspect = ss / ps[0]

# create 3D array
img_shape = list(slices[0].pixel_array.shape)
img_shape.append(len(slices))
img3d = np.zeros(img_shape)
m=0
# fill 3D array with the images from the files
for i, s in enumerate(slices):
#     print(s.pixel_array.shape,slices[0].pixel_array.shape)
    if s.pixel_array.shape != slices[0].pixel_array.shape:
        continue
    img2d = s.pixel_array
#     print(img2d)
    maximum=max(map(max,img2d))
    print(maximum)
    if maximum>m:
        m=maximum
    img3d[:, :, i] = img2d
print(m)


loading I0000471006.dcm
loading I0000470709.dcm
loading I0000470686.dcm
loading I0000470312.dcm
loading I0000470998.dcm
loading I0000470420.dcm
loading I0000470452.dcm
loading I0000470741.dcm
loading I0000470553.dcm
loading I0000470751.dcm
loading I0000470723.dcm
loading I0000470334.dcm
loading I0000470822.dcm
loading I0000470988.dcm
loading I0000470743.dcm
loading I0000470363.dcm
loading I0000470848.dcm
loading I0000472324.dcm
loading I0000470561.dcm
loading I0000470301.dcm
loading I0000470347.dcm
loading I0000470373.dcm
loading I0000470563.dcm
loading I0000470315.dcm
loading I0000470901.dcm
loading I0000470559.dcm
loading I0000470581.dcm
loading I0000470920.dcm
loading I0000470508.dcm
loading I0000470670.dcm
loading I0000470394.dcm
loading I0000470590.dcm
loading I0000470464.dcm
loading I0000470991.dcm
loading I0000470408.dcm
loading I0000470316.dcm
loading I0000470858.dcm
loading I0000470673.dcm
loading I0000470286.dcm
loading I0000470637.dcm
loading I0000470474.dcm
loading I0000470

6859
6763
7068
7109
7081
6262
6394
6423
6360
6414
6431
6474
6391
6525
6584
6428
6176
6162
6271
6275
6179
6259
6265
6207
6208
6180
6210
6167
6155
6152
6321
6423
6415
6270
6300
6167
6165
6139
6125
6221
6229
6090
6084
6150
6194
6246
6251
6201
6173
6224
6306
6308
6202
6151
6201
6162
6029
6119
6151
6093
6095
6089
6182
6269
6275
6168
6155
6173
6166
6384
6535
6173
6168
6204
6240
6109
6151
6061
6143
6147
6045
6058
6110
6104
6279
6232
6049
6113
6063
6033
6154
6015
5996
5964
5923
5902
5938
5950
6063
6054
6103
6166
6145
6070
6040
6060
6182
6196
6197
6037
6082
6105
6069
6074
6102
6141
6157
6162
6123
6143
6237
6291
6158
5951
5766
5824
5782
5807
5730
5805
5784
5802
5839
5804
5799
5810
5826
5855
5780
5828
5913
5942
5718
5816
5878
5827
5931
5853
5747
5676
5783
5785
5830
5773
5694
5727
5765
5867
5810
6085
6188
5940
5817
5900
5891
6043
6104
6042
5972
5919
5969
6284
6399
6421
6406
6330
6463
8358
9899
10153
10318
10726
10939
11858
11342
11484
11574
12199
12554
12513
12400
12459
12917
13344
13529
13269
129

In [19]:
def data_normalization(img3d):
    minimum=np.min(img3d)
    maximum=np.max(img3d)
    img3d=np.subtract(img3d,minimum)
    img3d=img3d / (maximum-minimum)
    return img3d

In [20]:
img3d=data_normalization(img3d)

0.0

In [12]:
ipv.pylab.clear()
ipv.pylab.plot_isosurface(img3d,level=3000)
ipv.pylab.style.set_style_dark()
ipv.pylab.view()
ipv.pylab.show()